In [3]:
from transformers import (BertTokenizer, AutoTokenizer, AutoModelForTokenClassification,
                          BertForTokenClassification, DataCollatorForTokenClassification, TrainingArguments,
                          Trainer)

from datasets import load_from_disk, load_metric

import os
import numpy as np

In [18]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

True
NVIDIA GeForce RTX 3060
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12050MB, multi_processor_count=28)


In [4]:
wiki_datasets = load_from_disk("/home/abdullah/Code/dl/lt_bert/dataset/wikiann")

metric = load_metric("seqeval")

wiki_datasets

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
})

In [5]:
MODEL_NAME = "/home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(tokenizer.is_fast)

inputs = tokenizer(wiki_datasets["train"][0]
                   ["tokens"], is_split_into_words=True)



True


In [7]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


In [8]:
labels = wiki_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))


[5, 6, 6]
[-100, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, -100]


In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs


In [10]:
tokenized_datasets = wiki_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=wiki_datasets["train"].column_names,
)


Loading cached processed dataset at /home/abdullah/Code/dl/lt_bert/dataset/wikiann/validation/cache-4422150ff2692120.arrow
100%|██████████| 1/1 [00:00<00:00, 12.59ba/s]
Loading cached processed dataset at /home/abdullah/Code/dl/lt_bert/dataset/wikiann/train/cache-e7e9c9e53542a14a.arrow


In [11]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

label_names = wiki_datasets["train"].features["ner_tags"].feature.names
labels = wiki_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels


['B-LOC', 'I-LOC', 'I-LOC']

In [12]:
predictions = labels.copy()
predictions[1] = "I-LOC"
metric.compute(predictions=[predictions], references=[labels])


{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [13]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100]
                   for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(
        predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


In [14]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    id2label=id2label,
    label2id=label2id,
)

model.config.num_labels

Some weights of the model checkpoint at /home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at

7

In [16]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

args = TrainingArguments(
    output_dir="temp",
    num_train_epochs=100,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=384,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=400,
    save_steps=400,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()


using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 10000
  Num Epochs = 100
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 5300
  8%|▊         | 400/5300 [00:19<04:13, 19.30it/s]***** Running Evaluation *****
  Num examples = 1000
  Batch size = 384


{'loss': 0.9667, 'learning_rate': 4e-05, 'epoch': 7.55}


                                                  
  8%|▊         | 400/5300 [00:20<04:13, 19.30it/s]Saving model checkpoint to temp/checkpoint-400
Configuration saved in temp/checkpoint-400/config.json
Model weights saved in temp/checkpoint-400/pytorch_model.bin
tokenizer config file saved in temp/checkpoint-400/tokenizer_config.json
Special tokens file saved in temp/checkpoint-400/special_tokens_map.json


{'eval_loss': 0.8111760020256042, 'eval_precision': 0.16948378254910917, 'eval_recall': 0.33514001806684734, 'eval_f1': 0.22512135922330098, 'eval_accuracy': 0.6978030746316294, 'eval_runtime': 0.507, 'eval_samples_per_second': 1972.389, 'eval_steps_per_second': 5.917, 'epoch': 7.55}


 15%|█▌        | 800/5300 [00:40<03:28, 21.57it/s]***** Running Evaluation *****
  Num examples = 1000
  Batch size = 384


...

                                                   
 98%|█████████▊| 5200/5300 [04:32<00:04, 21.58it/s]Saving model checkpoint to temp/checkpoint-5200
Configuration saved in temp/checkpoint-5200/config.json


{'eval_loss': 0.32981517910957336, 'eval_precision': 0.5772849462365591, 'eval_recall': 0.7759710930442638, 'eval_f1': 0.6620423892100192, 'eval_accuracy': 0.8930794191180382, 'eval_runtime': 0.4551, 'eval_samples_per_second': 2197.438, 'eval_steps_per_second': 6.592, 'epoch': 98.11}


Model weights saved in temp/checkpoint-5200/pytorch_model.bin
tokenizer config file saved in temp/checkpoint-5200/tokenizer_config.json
Special tokens file saved in temp/checkpoint-5200/special_tokens_map.json
100%|█████████▉| 5297/5300 [04:38<00:00, 20.29it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-5200 (score: 0.32981517910957336).
100%|██████████| 5300/5300 [04:38<00:00, 19.01it/s]

{'train_runtime': 278.7794, 'train_samples_per_second': 3587.065, 'train_steps_per_second': 19.011, 'train_loss': 0.5227531519475973, 'epoch': 100.0}


TrainOutput(global_step=5300, training_loss=0.5227531519475973, metrics={'train_runtime': 278.7794, 'train_samples_per_second': 3587.065, 'train_steps_per_second': 19.011, 'train_loss': 0.5227531519475973, 'epoch': 100.0})

In [17]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

args = TrainingArguments(
    output_dir="temp",
    num_train_epochs=100,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=384,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=400,
    save_steps=400,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=AutoModelForTokenClassification.from_pretrained("/home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert", id2label=id2label,
                                                          label2id=label2id,),
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()


using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
loading configuration file /home/abdullah/Code/dl/lt_bert/best_models/pruned_best_tiny_bert/config.json
Model config BertConfig {
  "_name_or_path": "google/bert_uncased_L-2_H-128_A-2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "

{'loss': 1.6435, 'learning_rate': 4e-05, 'epoch': 7.55}



  8%|▊         | 400/5300 [00:19<04:06, 19.88it/s]Saving model checkpoint to temp/checkpoint-400
Configuration saved in temp/checkpoint-400/config.json


{'eval_loss': 1.207398772239685, 'eval_precision': 0.028063851699279094, 'eval_recall': 0.0984643179765131, 'eval_f1': 0.04367862151873372, 'eval_accuracy': 0.4309271769774988, 'eval_runtime': 0.451, 'eval_samples_per_second': 2217.497, 'eval_steps_per_second': 6.652, 'epoch': 7.55}


Model weights saved in temp/checkpoint-400/pytorch_model.bin
tokenizer config file saved in temp/checkpoint-400/tokenizer_config.json
Special tokens file saved in temp/checkpoint-400/special_tokens_map.json
 15%|█▌        | 800/5300 [00:39<03:22, 22.18it/s]***** Running Evaluation *****
  Num examples = 1000
  Batch size = 384


...


 98%|█████████▊| 5200/5300 [04:34<00:04, 21.27it/s]Saving model checkpoint to temp/checkpoint-5200
Configuration saved in temp/checkpoint-5200/config.json
Model weights saved in temp/checkpoint-5200/pytorch_model.bin
tokenizer config file saved in temp/checkpoint-5200/tokenizer_config.json
Special tokens file saved in temp/checkpoint-5200/special_tokens_map.json


{'eval_loss': 0.457880437374115, 'eval_precision': 0.500329163923634, 'eval_recall': 0.6865401987353207, 'eval_f1': 0.5788271134805789, 'eval_accuracy': 0.8466407787648279, 'eval_runtime': 0.4574, 'eval_samples_per_second': 2186.32, 'eval_steps_per_second': 6.559, 'epoch': 98.11}


100%|█████████▉| 5298/5300 [04:39<00:00, 20.26it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-5200 (score: 0.457880437374115).
100%|██████████| 5300/5300 [04:39<00:00, 18.94it/s]

{'train_runtime': 279.8424, 'train_samples_per_second': 3573.44, 'train_steps_per_second': 18.939, 'train_loss': 0.6899432113935363, 'epoch': 100.0}


TrainOutput(global_step=5300, training_loss=0.6899432113935363, metrics={'train_runtime': 279.8424, 'train_samples_per_second': 3573.44, 'train_steps_per_second': 18.939, 'train_loss': 0.6899432113935363, 'epoch': 100.0})